# import 

In [1]:
import pandas as pd 
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt

import uszipcode
from uszipcode import SearchEngine

import plotly as plty


/usr/local/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# data

Variable Name	    Description

LoanNr_ChkDgt	    Identifier Primary key

Name	            Borrower name

City	            Borrower city

State	            Borrower state

Zip	                Borrower zip code

Bank	            Bank name

BankState	        Bank state

NAICS	            North American industry classification system code

ApprovalDate	    Date SBA commitment issued

ApprovalFY	        Fiscal year of commitment

Term	            Loan term in months

NoEmp	            Number of business employees

NewExist	        1 = Existing business, 2 = New business

CreateJob	        Number of jobs created

RetainedJob	        Number of jobs retained

FranchiseCode	    Franchise code, (00000 or 00001) = No franchise

UrbanRural	        1 = Urban, 2 = rural, 0 = undefined

RevLineCr	        Revolving line of credit: Y = Yes, N = No
    
LowDoc	            LowDoc Loan Program: Y = Yes, N = No
    
ChgOffDate	        The date when a loan is declared to be in default

DisbursementDate	Disbursement date

DisbursementGross	Amount disbursed

BalanceGross	    Gross amount outstanding

MIS_Status	        Loan status charged off = CHGOFF, Paid in full =PIF

ChgOffPrinGr	    Charged-off amount

GrAppv	            Gross amount of loan approved by bank

SBA_Appv	        SBA’s guaranteed amount of approved loan

In [2]:
df = pd.read_csv('explore.csv',index_col= 0 )

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_43912/711269991.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('explore.csv',index_col= 0 )


In [3]:
def camel_to_snake(column_name):
    """Converts a camel case string to snake case"""
    snake_name = ''
    for i, char in enumerate(column_name):
        if i == 0:
            snake_name += char.lower()
        elif char.isupper():
            snake_name += '_' + char.lower()
        else:
            snake_name += char
    return snake_name

In [4]:
snake_case_columns = [camel_to_snake(col) for col in df]

In [5]:
snake_case_columns

['city',
 'state',
 'zip',
 'bank',
 'bank_state',
 'n_a_i_c_s',
 'approval_date',
 'approval_f_y',
 'term',
 'no_emp',
 'new_exist',
 'create_job',
 'retained_job',
 'franchise_code',
 'urban_rural',
 'rev_line_cr',
 'low_doc',
 'chg_off_date',
 'disbursement_date',
 'disbursement_gross',
 'balance_gross',
 'm_i_s__status',
 'chg_off_prin_gr',
 'gr_appv',
 's_b_a__appv']

In [6]:
# lis = df.columns.tolist()

In [7]:
# lis = [x.lower() for x in lis]

In [8]:
df.columns = snake_case_columns

In [9]:
df = df.drop(columns = ['approval_date','chg_off_date','disbursement_date'])

# Explore

In [10]:
df=df.rename(columns = {'m_i_s__status':'target'})

In [11]:
state_point = df.groupby('state')['target'].value_counts(normalize=True).unstack()['P I F'].sort_values(ascending = False)

In [12]:
#Next, we will give points to each State.
#I indicated only the highest and the lowest default rates in the library and assigned def rate = 18 (mean) to the states left
#Function for the state scores 
def apply_score_state(i):
    state_default = {'MT':8, 'ND': 8, 'WY':8, 'SD':8, 'VT':8, 'ME':10,'NH':10, 'NM':10, 'AK':10, 'WA':13,'AD':13, 'MN':13, 
                     'WI':13, 'IA':13,'NE':13, 'KS':13, 'MA':13,'CT':13,'RI':13,'PA':13, 'NV':23, 'IL':23, 'MI':23, 'KY':23,
                     'GA':23, 'FL':28}
    temp_defrate = None
    average_def_rate = 18
    if i in state_default:
        temp_defrate = state_default[i]
        return point_def(temp_defrate)
    return point_def(average_def_rate)

In [13]:
state_point= pd.DataFrame(state_point)

In [14]:
state_point['P I F']

state
MT    0.931319
WY    0.929365
VT    0.925788
ND    0.922676
SD    0.921591
ME    0.902014
NH    0.892258
NM    0.891660
NE    0.887228
AK    0.886017
RI    0.885224
IA    0.883498
MN    0.882677
WI    0.877927
MA    0.869755
KS    0.869473
WA    0.859606
CT    0.859428
ID    0.857421
PA    0.853048
HI    0.847434
MO    0.847055
OR    0.846717
OK    0.844662
MS    0.841583
WV    0.838190
OH    0.834974
AL    0.834020
AR    0.831060
UT    0.823545
IN    0.823381
CO    0.821207
DE    0.820916
LA    0.819021
CA    0.814620
VA    0.812128
TX    0.810971
NC    0.807452
KY    0.805198
MD    0.801271
NY    0.799655
NJ    0.798211
SC    0.797602
AZ    0.792553
TN    0.788243
MI    0.773729
IL    0.772263
NV    0.767319
DC    0.761101
GA    0.760205
FL    0.724731
Name: P I F, dtype: float64

In [15]:
state_score = {}
for i in range(0,len(state_point)):
    
    if state_point.iloc[i][0] > .90:
        
        dict1 = {f'{state_point.iloc[i].name}': 1}
        
        state_score.update(dict1)
    
    elif state_point.iloc[i][0] > .86:
        
        dict1 = {f'{state_point.iloc[i].name}': 2}
        
        state_score.update(dict1)
    
    elif state_point.iloc[i][0] > .76:
        
        dict1 = {f'{state_point.iloc[i].name}': 3}
        
        state_score.update(dict1)
    
    else:
        
        dict1 = {f'{state_point.iloc[i].name}': 4}
        
        state_score.update(dict1)
        

        # state_score.update(f'state_point.iloc[i].name', '90')
        
    
    
    
 

In [16]:
state_score

{'MT': 1,
 'WY': 1,
 'VT': 1,
 'ND': 1,
 'SD': 1,
 'ME': 1,
 'NH': 2,
 'NM': 2,
 'NE': 2,
 'AK': 2,
 'RI': 2,
 'IA': 2,
 'MN': 2,
 'WI': 2,
 'MA': 2,
 'KS': 2,
 'WA': 3,
 'CT': 3,
 'ID': 3,
 'PA': 3,
 'HI': 3,
 'MO': 3,
 'OR': 3,
 'OK': 3,
 'MS': 3,
 'WV': 3,
 'OH': 3,
 'AL': 3,
 'AR': 3,
 'UT': 3,
 'IN': 3,
 'CO': 3,
 'DE': 3,
 'LA': 3,
 'CA': 3,
 'VA': 3,
 'TX': 3,
 'NC': 3,
 'KY': 3,
 'MD': 3,
 'NY': 3,
 'NJ': 3,
 'SC': 3,
 'AZ': 3,
 'TN': 3,
 'MI': 3,
 'IL': 3,
 'NV': 3,
 'DC': 3,
 'GA': 3,
 'FL': 4}

In [17]:
df.state[1] in state_score

True

In [18]:
state_score['GA']

3

In [19]:
def apply_score(state):
    
    if state in state_score:
        return(state_score[state])
    
    
        
        

In [20]:
# create a new column called 'n_a_i_c_s_first_two'
df['sec_points'] = df['n_a_i_c_s'].apply(lambda x: int(str(x)[:2]))

In [21]:
df['state_score'] = df.state.apply(apply_score)

In [22]:
# drop columns / city : to many vars, state: replaced by state score.
df.drop(columns = ['city','state','bank_state','approval_f_y','bank','n_a_i_c_s'], inplace=True)

In [23]:
df['target'] = df['target'].replace({'P I F': 0, 'CHGOFF': 1})

In [26]:
df['rev_line_cr'] = df['rev_line_cr'].replace({'N': 0, 'Y': 1})

In [27]:
df['low_doc'] = df['low_doc'].replace({'N': 0, 'Y': 1})

In [29]:
df

,zip,term,no_emp,new_exist,create_job,retained_job,franchise_code,urban_rural,rev_line_cr,low_doc,disbursement_gross,balance_gross,target,chg_off_prin_gr,gr_appv,s_b_a__appv,sec_points,state_score
0,47711,84,4,2,0,0,1,0,0,1,60000,0,0,0,60000,48000,45,3
1,46526,60,2,2,0,0,1,0,0,1,40000,0,0,0,40000,32000,72,3
2,47401,180,7,1,0,0,1,0,0,0,287000,0,0,0,287000,215250,62,3
3,74012,60,2,1,0,0,1,0,0,1,35000,0,0,0,35000,28000,0,3
4,32801,240,14,1,7,7,1,0,0,0,229000,0,0,0,229000,229000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,43221,60,6,1,0,0,1,0,0,0,70000,0,0,0,70000,56000,45,3
899160,43221,60,6,1,0,0,1,0,1,0,85000,0,0,0,85000,42500,45,3
899161,93455,108,26,1,0,0,1,0,0,0,300000,0,0,0,300000,225000,33,3
899162,96830,60,6,1,0,0,1,0,0,1,75000,0,1,46383,75000,60000,0,3
